In [1]:
import os
import sys
# import yfinance as yf
# import yfinance.shared as shared
import pandas as pd

current_dir = os.getcwd()
# Parent directory where myUtils is located
path_utils = os.path.dirname(current_dir)
sys.path.append(path_utils)


from myUtils import yf_download_AdjOHLCV_noAutoAdj, adjust_OHLC, yf_candlestick
# from myUtils import yf_download_AdjOHLCV_noAutoAdj, adjust_OHLC


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
days_plot = 252 * 2

In [3]:
plt_syms =   ["ETN", "VOO"]

# verify df plt_syms' close against Yahoo
# even if discrepancy, the relative relationship of OHLCV in df_close matches Yahoo_Close
df_list=[]
df = yf_download_AdjOHLCV_noAutoAdj(plt_syms, verbose=False)
df_idx = yf_download_AdjOHLCV_noAutoAdj("XOM", verbose=False)

# Change the DatetimeIndex to "yyyy-mm-dd" format
df.index = df.index.strftime('%Y-%m-%d')
df_idx.index = df_idx.index.strftime('%Y-%m-%d')

# sets df's index to df_idx's index, this prevent use of weekend data
#  like BTC-USD which trades in weekends
df = df.loc[df_idx.index]

df_list.append(df)
df = pd.concat(df_list, axis=1)
# get unique symbols from column name
_l_symbols = list(set(df.columns.get_level_values(0)))
_l_symbols.sort()

df_adj_list=[]
# adjust OHLC using 'Adj Close'
for symbol in _l_symbols:
  df1 = adjust_OHLC(df[symbol])
  df_adj_list.append(df1)

df_adj = pd.concat(df_adj_list, axis=1)
_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
# create multilevel column names
_col_names = pd.MultiIndex.from_product([_l_symbols, _cols])
df_adj.columns = _col_names

[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  1 of 1 completed


In [4]:
df = df_adj["ETN"].iloc[-days_plot:]
close = df.Close.values
volume = df.Volume.values.astype(float)
df

,Open,High,Low,Close,Volume
Date,,,,,
2022-09-20,134.415537,135.480259,133.205633,134.715591,1621600
2022-09-21,136.099716,137.454803,133.370165,133.408890,1112700
2022-09-22,132.537768,133.341146,130.214757,130.795517,1871600
2022-09-23,129.556558,130.282501,128.017549,129.837250,1726400
2022-09-26,129.595295,131.114942,128.143409,129.246841,1674200
...,...,...,...,...,...
2024-09-16,306.000000,306.980011,301.480011,306.549988,1536300
2024-09-17,308.480011,312.420013,304.649994,311.730011,2117000
2024-09-18,313.559998,321.149994,313.549988,314.649994,2931000


In [5]:
# ++ On-Balance-Volume ++
# OBV_period = 10  # OBV's EMA period
OBV_period = 50  # OBV's EMA period
EMA_pd=OBV_period
tail_pd=5
norm_pd=30

In [ ]:
print(f'close:\n{close[-10:]}')

In [ ]:
print(f'volume:\n{volume[-10:]}')

In [ ]:
for item in OBV_EMA:
  print(item)

In [6]:
import talib as ta
import numpy as np

# close = df.close.values  # convert dataframe to numpy array
close = df.Close.values  # convert dataframe to numpy array
# .astype(float) converts df.volume from integer to float for talib
volume = df.Volume.values.astype(float)
OBV = ta.OBV(close, volume)
OBV_EMA = ta.EMA(OBV, timeperiod=EMA_pd)  # calculate EMA of OBV
# average OBV_EMA difference for the last tail_pd days, default is 5 days
# avg_OBV_EMA_dif = np.mean(np.diff(OBV_EMA[-(tail_pd+1):]))
# avg_OBV_EMA_dif = np.mean(np.diff(OBV_EMA[-(tail_pd+1):]))
OBV_dif = (OBV[-1] - OBV[-(tail_pd+1)]) / tail_pd
# average volume for last norm_pd days, default is 30 days
avg_volume = np.mean(volume[-norm_pd:])
# normalized avg_OBV_EMA_dif by avg_volume
# OBV_slope = avg_OBV_EMA_dif / avg_volume
OBV_slope = OBV_dif / avg_volume

In [8]:
OBV_tail = OBV[-(tail_pd+1):]
lin_reg_slope = ta.LINEARREG_SLOPE(OBV_tail, tail_pd+1)[-1]
OBV_lin_reg_slope = lin_reg_slope / avg_volume

print(f'OBV_tail: {OBV_tail}')
print(f'lin_reg_slope: {lin_reg_slope}')
print(f'avg_volume: {avg_volume}')
print(f'OBV_lin_reg_slope: {OBV_lin_reg_slope}')
print(f'OBV_slope: {OBV_slope}')

OBV_tail: [56454800. 57991100. 60108100. 63039100. 66194800. 69826200.]
lin_reg_slope: 2697117.1428571427
avg_volume: 2166813.3333333335
OBV_lin_reg_slope: 1.2447390374453773
OBV_slope: 1.2341995311086633


In [17]:
# TODO this is new
def OBV_calc(df, symbol, EMA_pd=50, tail_pd=5, norm_pd=30):
    """Calculate On-Balance-Volume(OBV), EMA of OBV,
       normalized EMA slope of the On-Balance-Volume (OBV).

    Args:
        df(dataframe): dataframe with date index
                       and columns open, high, low, close, volume
        symbol(str): symbol, e.g. 'QQQ'
        EMA_pd(int): exponential moving average period of OBV
        tail_pd(int): number of periods used to calculate avg_OBV_EMA_dif
        norm_pd(int): number of periods used to calculate avg_volume

    Return:
        OBV(numpy.ndarray): on-balance-volume
        OBV_EMA(numpy.ndarray): EMA of OBV with time-period=EMA_pd
        OBV_slope(float): lin_regr_slope / avg_volume,
          (lin_regr_slope is lope of linear regression line of OBV_tail array),
          (avg_volume is the average volume for last norm_pd days, default is 30 days)
    """
    # df.Close.values was df.close.values
    # df.Volume.values.astype(float) was df.volume.values.astype(float)
    # 2024-09-21 default EMA_pd was 10  
    # 2024-09-22 calculate OBV_slope with lin_regr_slope  


    import talib as ta
    import numpy as np


    close = df.Close.values  # convert dataframe to numpy array

    # .astype(float) converts df.volume from integer to float for talib
    # volume = df.volume.values.astype(float)
    volume = df.Volume.values.astype(float)

    # Calculate on-balance-volume
    OBV = ta.OBV(close, volume)

    # get the tail of OBV array
    OBV_tail = OBV[-(tail_pd+1):]

    OBV_EMA = ta.EMA(OBV, timeperiod=EMA_pd)  # calculate EMA of OBV
    
    # slope of linear regression line of OBV_tail array
    lin_regr_slope = ta.LINEARREG_SLOPE(OBV_tail, tail_pd+1)[-1]

    # average volume for last norm_pd days, default is 30 days
    avg_volume = np.mean(volume[-norm_pd:])

    # normalize lin_regr_slope to avg_volume
    OBV_slope = lin_regr_slope / avg_volume

    return OBV, OBV_EMA, OBV_slope


In [33]:
def yf_candlestick(symbol, df, plot_chart=True):
    """Plot candlestick chart and returns:
        symbol, date, UI_MW_short[-1], UI_MW_long[-1],
        diff_UI_MW_short[-1], diff_UI_MW_long[-1]

        revised 2024-09-02
        from: date = df.index[-1].strftime("%Y-%m-%d")
        to:   date = df.index[-1]  # df index is in "yyyy-mm-dd" format 
        

    Args:
        symbol(string): symbol for the dataframe df
        df(dataframe): dataframe with date index
                       and columns open, high, low, close, volume
        plot_chart(bool): plots chart if True, default is True

    Return:
        cache(tuple) of:
            symbol(str): stock symbol, e.g. 'AMZN'
            date(str): plot's last date, e.g. '2018-01-05'
            UI_MW_short[-1](float): last value of UI_MW_short
            UI_MW_long[-1](float): last value of UI_MW_long
            diff_UI_MW_short[-1](float): last value of diff_UI_MW_short
            diff_UI_MW_long[-1](float): last value of diff_UI_MW_long
    """
    # 2022-08-21 change OBV's EMA period from 10 to 50, i.e.  OBV_period = 50
    # 2023-02-28 change from df.close.values to df.Close.values 


    # Reference:
    # https://www.kaggle.com/arrowsx/crypto-currency-a-plotly-sklearn-tutorial
    # https://plotly.com/python/candlestick-charts/
    # http://mrjbq7.github.io/ta-lib/
    # https://www.somesolvedproblems.com/2018/10/how-to-customize-plotlys-modebar.html
    # https://plotly.com/python-api-reference/generated/plotly.graph_objects.Candlestick.html#plotly.graph_objects.Candlestick
    # https://plotly.com/python-api-reference/generated/plotly.graph_objects.layout.html
    # https://community.plotly.com/t/random-vertical-lines-extending-to-x-axis-on-line-chart/23462
    # https://plotly.com/python/hover-text-and-formatting/

    from plotly.offline import plot


    # from myUtils import symb_perf_stats, OBV_calc, UI_MW
    from myUtils import symb_perf_stats, UI_MW


    import plotly.graph_objs as go
    import talib as ta
    import numpy as np

    # **** IMPORTANT: COMMENT-OUT LINES INCLUDING THE BRACKETS {...} ****
    # if no lines are between the brackets, chart will be plotted compressed
    # to the right side and leaving empty spaces on the left side
    # ********

    chart_path = "C:/Users/ping/Desktop/plotly/"  # directory to store charts
    chart_name = symbol + ".html"


    # close = df.close.values  # convert dataframe to numpy array
    close = df.Close.values  # convert dataframe to numpy array
    volume = df.Volume.values  # convert dataframe to numpy array

    # Calculate the average of the last 30 elements
    avg_volume_30 = np.mean(volume[-30:])  

    # +++++++++ panel layout ++++++++++
    x_annotation = 0.06  # text x position
    y_annotation_gap = 0.022  # text y spacing

    # panels' y positions
    y80_top = 0.99  # spike, height 1
    y80_btm = 0.98
    y70_top = 0.98  # price, height 38
    y70_txt = 0.96  # price, text
    y70_btm = 0.60
    # gap of 1
    y60_top = 0.59  # volume, height 5
    # y60_txt = .57  # volume, text
    y60_btm = 0.54
    # gap of 1
    y50_top = 0.53  # drawdown, height 5
    y50_txt = 0.52  # drawdown, text
    y50_btm = 0.48
    # gap of 1
    y40_top = 0.47  # OBV, height 5
    y40_txt = 0.46  # OBV, text
    y40_btm = 0.42
    # gap of 1
    y30_top = 0.41  # top, height 13
    y30_txt = 0.40  # top, text
    y30_btm = 0.28
    # gap of 1
    y20_top = 0.27  # middle, height 13
    y20_txt = 0.26  # middle, text
    y20_btm = 0.14
    # gap of 1
    y10_top = 0.13  # bottom, height 13
    y10_txt = 0.12  # bottom, text
    y10_btm = 0.0

    # +++++++++ chart colors ++++++++++
    # http://colormind.io/template/paper-dashboard/#
    INCREASING_COLOR = "#17BECF"
    DECREASING_COLOR = "#7F7F7F"
    BBANDS_COLOR = "#BDBCBC"
    LIGHT_COLOR = "#0194A2"
    DARK_COLOR = "#015C65"
    DRAWDOWN_COLOR = "#5B5170"

    # ++++++++++ moving window sizes +++++++++++
    window_short = 15
    window_long = 30

    # ++++++++++ indicators +++++++++++
    # ++ spike line ++
    # With Toggle Spike Lines is on, gives vertical lines down the chart
    #   when cursor is at top of chart
    spike_line = np.zeros(len(df.index))  # create a trace with zeros
    spike_line_color = DRAWDOWN_COLOR

    # ++ moving averages ++
    EMA_fast_period = 50  # moving average rolling window width
    EMA_fast_label = "EMA" + str(EMA_fast_period)  # moving average column name
    EMA_fast_color = LIGHT_COLOR
    EMA_fast = ta.EMA(close, timeperiod=EMA_fast_period)

    EMA_slow_period = 200  # moving average rolling window width
    EMA_slow_label = "EMA" + str(EMA_slow_period)  # moving average column name
    EMA_slow_color = DARK_COLOR
    EMA_slow = ta.EMA(close, timeperiod=EMA_slow_period)

    # ++ Bollinger Bands ++
    BBands_period = 20
    BBands_stdev = 2
    BBands_label = (
        "BBands(" + str(BBands_period) + "," + str(BBands_stdev) + ")"
    )
    BB_upper_color = BBANDS_COLOR
    BB_avg_color = BBANDS_COLOR
    BB_lower_color = BBANDS_COLOR
    BB_upper, BB_avg, BB_lower = ta.BBANDS(
        close,
        timeperiod=BBands_period,
        nbdevup=BBands_stdev,
        nbdevdn=BBands_stdev,
        matype=0,
    )  # simple-moving-average


########################################
# def OBV_calc(df, symbol, EMA_pd=10, tail_pd=5, norm_pd=30):
    # ++ On-Balance-Volume ++
    # OBV_period = 10  # OBV's EMA period
    # OBV_period = 50  # OBV's EMA period
    # OBV, OBV_EMA, OBV_slope = OBV_calc(
    #     df, symbol, EMA_pd=OBV_period, tail_pd=5, norm_pd=30
    # )
    OBV, OBV_EMA, OBV_slope = OBV_calc(df, symbol)
########################################


    # ++ On-Balance-Volume Difference ++
    OBV_diff = OBV - OBV_EMA

    # ++ UI M.W., Ulcer-Index Moving-Window ++
    # get drawdown array
    (
        period_yr,
        CAGR,
        CAGR_Std,
        CAGR_UI,
        daily_return_std,
        Std_UI,
        drawdown,
        ulcer_index,
        max_drawdown,
    ) = symb_perf_stats(close)
    # calculate Ulcer_Index of moving-window applied to drawdown
    UI_MW_short = UI_MW(drawdown, window=window_short)
    UI_MW_long = UI_MW(drawdown, window=window_long)

    # ++ Diff. UI M.W., difference in UI_MW_short and long arrays ++
    diff_UI_MW_short = np.diff(UI_MW_short)
    diff_UI_MW_long = np.diff(UI_MW_long)
    # num of leading NaN needed to pad arrays to the same length as 'close'
    num_NA_pad_short = len(close) - len(diff_UI_MW_short)
    num_NA_pad_long = len(close) - len(diff_UI_MW_long)
    # add leading NaN pad to the arrays
    diff_UI_MW_short = np.pad(
        diff_UI_MW_short,
        (num_NA_pad_short, 0),
        "constant",
        constant_values=(np.nan),
    )
    diff_UI_MW_long = np.pad(
        diff_UI_MW_long,
        (num_NA_pad_long, 0),
        "constant",
        constant_values=(np.nan),
    )

    # +++++ Cache +++++
    # change date index from 2018-09-04 00:00:00 to 2018-09-04
    #   dtype also change from datetime64[ns] to object

###############################################################################
    # date = df.index[-1].strftime("%Y-%m-%d")
    date = df.index[-1]  # df index is in "yyyy-mm-dd" format 
    
    cache = (
        symbol,
        date,
        UI_MW_short[-1],
        UI_MW_long[-1],
        diff_UI_MW_short[-1],
        diff_UI_MW_long[-1],
    )

    # +++++ Set Volume Bar Colors +++++
    colors_volume_bar = []

    for i, _ in enumerate(df.index):
        if i != 0:
            # if df["close"].iloc[i] > df["close"].iloc[i - 1]:
            if df["Close"].iloc[i] > df["Close"].iloc[i - 1]:
                colors_volume_bar.append(INCREASING_COLOR)
            else:
                colors_volume_bar.append(DECREASING_COLOR)
        else:
            colors_volume_bar.append(DECREASING_COLOR)

    # +++++ Set OBV_diff Bar Colors +++++
    colors_OBV_diff_bar = []

    for value in OBV_diff:
        if value > 0:
            colors_OBV_diff_bar.append(INCREASING_COLOR)
        else:
            colors_OBV_diff_bar.append(DECREASING_COLOR)

    # +++++++ panel definition ++++++++
    # ++ drawdown panel ++
    DD_title = "DD"
    DD_label = DD_title
    DD_panel_text1 = "UI: " + str("%.3f" % ulcer_index)
    DD_panel_text2 = "MaxDD: " + str("%.3f" % max_drawdown)
    DD_panel_trace1 = drawdown
    DD_trace1_color = DRAWDOWN_COLOR
    DD_trace2_color = None

    # ++ OBV panel ++
    OBV_title = "OBV"
    OBV_label = "OBV"
    OBV_EMA_label = "EMA" + str(OBV_period)
    OBV_panel_text1 = "OBV Slope(" + str(tail_pd) + "): " + "%.3f" % OBV_slope
    OBV_panel_text2 = "Avg Volume (30d): " + "%.3f" % (avg_volume_30 / 1E6) + "M"       
    OBV_panel_trace1 = OBV
    OBV_trace1_color = LIGHT_COLOR
    OBV_panel_trace2 = OBV_EMA
    OBV_trace2_color = DARK_COLOR

    # ++ top panel ++
    top_title = "Diff. OBV"
    top_panel_text1 = OBV_label + "-" + OBV_EMA_label
    top_panel_trace1 = OBV_diff
    top_trace1_color = LIGHT_COLOR

    # ++ middle panel ++
    mid_title = "UI MW"
    mid_panel_text1 = "MW" + str(window_short)
    mid_panel_text2 = "MW" + str(window_long)
    mid_panel_trace1 = UI_MW_short
    mid_panel_trace2 = UI_MW_long
    mid_trace1_color = LIGHT_COLOR
    mid_trace2_color = DARK_COLOR

    # ++ bottom panel ++
    btm_title = "Diff. UI M.W."
    btm_panel_text1 = "MW" + str(window_short)
    btm_panel_text2 = "MW" + str(window_long)
    btm_panel_trace1 = diff_UI_MW_short
    btm_panel_trace2 = diff_UI_MW_long
    btm_trace1_color = LIGHT_COLOR
    btm_trace2_color = DARK_COLOR

    # +++++++ plotly layout ++++++++
    layout = {
        # borders
        "margin": {
            "t": 20,
            "b": 20,
            "r": 20,
            "l": 80,
        },
        "annotations": [
            # symbol
            {
                "x": 0.25,
                "y": 0.998,
                "showarrow": False,
                "text": "<b>" + symbol + "</b>",  # bold
                "xref": "paper",
                "yref": "paper",
                "xanchor": "left",
                "yanchor": "bottom",
                "font": {
                    "size": 15,  # larger font
                },
            },
            # period
            {
                "x": x_annotation,
                "y": y70_txt,  # 1st line
                "text": "Years: " + "%.2f" % period_yr,
                "xref": "paper",
                "yref": "paper",
                "xanchor": "left",
                "yanchor": "middle",
                "showarrow": True,
                "arrowhead": 0,
                "arrowwidth": 1,
                "arrowcolor": DECREASING_COLOR,
                "ax": -60,
                "ay": 0,
                "font": {
                    "size": 12,  # default font size is 12
                },
            },
            # Std(Daily Returns) / Ulcer_Index, Smaller is better
            {
                "x": x_annotation,
                "y": y70_txt - 1 * y_annotation_gap,  # 2nd line
                "text": "Std/UI: " + "%.3f" % Std_UI,
                "xref": "paper",
                "yref": "paper",
                "xanchor": "left",
                "yanchor": "middle",
                "showarrow": True,
                "arrowhead": 0,
                "arrowwidth": 1,
                "arrowcolor": DECREASING_COLOR,
                "ax": -60,
                "ay": 0,
                "font": {
                    "size": 12,  # default font size is 12
                },
            },
            # CAGR
            {
                "x": x_annotation,
                "y": y70_txt - 2 * y_annotation_gap,  # 3rd line
                "text": "CAGR: " + "%.3f" % CAGR,
                "xref": "paper",
                "yref": "paper",
                "xanchor": "left",
                "yanchor": "middle",
                "showarrow": True,
                "arrowhead": 0,
                "arrowwidth": 1,
                "arrowcolor": DECREASING_COLOR,
                "ax": -60,
                "ay": 0,
                "font": {
                    "size": 12,  # default font size is 12
                },
            },
            # CAGR / Std
            {
                "x": x_annotation,
                "y": y70_txt - 3 * y_annotation_gap,  # 4th line
                "text": "CAGR/Std: " + "%.3f" % CAGR_Std,
                "xref": "paper",
                "yref": "paper",
                "xanchor": "left",
                "yanchor": "middle",
                "showarrow": True,
                "arrowhead": 0,
                "arrowwidth": 1,
                "arrowcolor": DECREASING_COLOR,
                "ax": -60,
                "ay": 0,
                "font": {
                    "size": 12,  # default font size is 12
                },
            },
            # CAGR / UI
            {
                "x": x_annotation,
                "y": y70_txt - 4 * y_annotation_gap,  # 5th line
                "text": "CAGR/UI: " + "%.3f" % CAGR_UI,
                "xref": "paper",
                "yref": "paper",
                "xanchor": "left",
                "yanchor": "middle",
                "showarrow": True,
                "arrowhead": 0,
                "arrowwidth": 1,
                "arrowcolor": DECREASING_COLOR,
                "ax": -60,
                "ay": 0,
                "font": {
                    "size": 12,  # default font size is 12
                },
            },
            # drawdown panel text1 line
            {
                "x": x_annotation,
                "y": y50_txt,
                "text": DD_panel_text1,
                "xref": "paper",
                "yref": "paper",
                "xanchor": "left",
                "yanchor": "middle",
                "showarrow": True,
                "arrowhead": 0,
                "arrowwidth": 1,
                "arrowcolor": DD_trace1_color,
                "ax": -60,
                "ay": 0,
            },
            # drawdown panel text2 line
            {
                "x": x_annotation,
                "y": y50_txt - y_annotation_gap,
                "text": DD_panel_text2,
                "xref": "paper",
                "yref": "paper",
                "xanchor": "left",
                "yanchor": "middle",
                "showarrow": True,
                "arrowhead": 0,
                "arrowwidth": 1,
                "arrowcolor": DD_trace2_color,
                "ax": -60,
                "ay": 0,
            },
            # OBV panel text1 line
            {
                "x": x_annotation,
                "y": y40_txt,
                "text": OBV_panel_text1,
                "xref": "paper",
                "yref": "paper",
                "xanchor": "left",
                "yanchor": "middle",
                "showarrow": True,
                "arrowhead": 0,
                "arrowwidth": 1,
                "arrowcolor": OBV_trace1_color,
                "ax": -60,
                "ay": 0,
            },
            # OBV panel text2 line
            {
                "x": x_annotation,
                "y": y40_txt - y_annotation_gap,
                "text": OBV_panel_text2,
                "xref": "paper",
                "yref": "paper",
                "xanchor": "left",
                "yanchor": "middle",
                "showarrow": True,
                "arrowhead": 0,
                "arrowwidth": 1,
                "arrowcolor": OBV_trace1_color,
                "ax": -60,
                "ay": 0,
            },



            # top panel text1 line
            {
                "x": x_annotation,
                "y": y30_txt,
                "text": top_panel_text1,
                "xref": "paper",
                "yref": "paper",
                "xanchor": "left",
                "yanchor": "middle",
                "showarrow": True,
                "arrowhead": 0,
                "arrowwidth": 1,
                "arrowcolor": top_trace1_color,
                "ax": -60,
                "ay": 0,
            },
            # middle panel text1 line
            {
                "x": x_annotation,
                "y": y20_txt,
                "text": mid_panel_text1,
                "xref": "paper",
                "yref": "paper",
                "xanchor": "left",
                "yanchor": "middle",
                "showarrow": True,
                "arrowhead": 0,
                "arrowwidth": 1,
                "arrowcolor": mid_trace1_color,
                "ax": -60,
                "ay": 0,
            },
            # middle panel text2 line
            {
                "x": x_annotation,
                "y": y20_txt - y_annotation_gap,
                "text": mid_panel_text2,
                "xref": "paper",
                "yref": "paper",
                "xanchor": "left",
                "yanchor": "middle",
                "showarrow": True,
                "arrowhead": 0,
                "arrowwidth": 1,
                "arrowcolor": mid_trace2_color,
                "ax": -60,
                "ay": 0,
            },
            # bottom panel text1
            {
                "x": x_annotation,
                "y": y10_txt,
                "text": btm_panel_text1,
                "xref": "paper",
                "yref": "paper",
                "xanchor": "left",
                "yanchor": "middle",
                "showarrow": True,
                "arrowhead": 0,
                "arrowwidth": 1,
                "arrowcolor": btm_trace1_color,
                "ax": -60,
                "ay": 0,
            },
            # bottom panel text2
            {
                "x": x_annotation,
                "y": y10_txt - y_annotation_gap,
                "text": btm_panel_text2,
                "xref": "paper",
                "yref": "paper",
                "xanchor": "left",
                "yanchor": "middle",
                "showarrow": True,
                "arrowhead": 0,
                "arrowwidth": 1,
                "arrowcolor": btm_trace2_color,
                "ax": -60,
                "ay": 0,
            },
        ],
        "xaxis": {
            # vertical spike line
            "showspikes": True,
            "spikemode": "toaxis+across",
            "spikedash": "solid",
            "spikethickness": 1,
            "spikecolor": spike_line_color,
            # range slider bar
            "rangeslider": {
                "visible": False  # removes rangeslider panel at bottom
            },
            # range buttons to interact
            "rangeselector": {
                "visible": True,
                "bgcolor": "rgba(150, 200, 250, 0.4)",  # background color
                "x": 0,
                "y": 1,
                "buttons": [
                    {"count": 1, "label": "reset", "step": "all"},
                    {
                        "count": 1,
                        "label": "1 yr",
                        "step": "year",
                        "stepmode": "backward",
                    },
                    {
                        "count": 6,
                        "label": "6 mo",
                        "step": "month",
                        "stepmode": "backward",
                    },
                    {
                        "count": 3,
                        "label": "3 mo",
                        "step": "month",
                        "stepmode": "backward",
                    },
                    {
                        "count": 1,
                        "label": "1 mo",
                        "step": "month",
                        "stepmode": "backward",
                    },
                    {
                        "count": 1,
                        "label": "ytd",
                        "step": "year",
                        "stepmode": "todate",
                    },
                ],
            },
        },
        # panel legends
        "legend": {
            "orientation": "h",
            "y": 0.99,
            "x": 0.3,
            "yanchor": "bottom",
        },
        # panel spike line
        "yaxis80": {
            "domain": [y80_btm, y80_top],
            "showticklabels": False,
            "showline": True,
        },
        # panel price
        "yaxis70": {
            "domain": [y70_btm, y70_top],
            "showticklabels": True,
            "showline": True,
            "title": "Price",
            "type": "log",
        },
        # panel volume
        "yaxis60": {
            "domain": [y60_btm, y60_top],
            "showticklabels": True,
            "showline": True,
            "title": "Vol.",
            "titlefont": {"size": 12},
            # 'type': 'log',
        },
        # panel drawdown
        "yaxis50": {
            "domain": [y50_btm, y50_top],
            "showticklabels": True,
            "showline": True,
            "title": DD_title,
            "titlefont": {"size": 12},
            # 'type': 'log',
        },
        # panel OBV
        "yaxis40": {
            "domain": [y40_btm, y40_top],
            "showticklabels": True,
            "showline": True,
            "title": OBV_title,
            "titlefont": {"size": 12},
            # 'type': 'log',
        },
        # panel top
        "yaxis30": {
            "domain": [y30_btm, y30_top],
            "showticklabels": True,
            "showline": True,  # vertical line on right side
            "title": top_title,
            "titlefont": {"size": 12},
            "tickmode": "array",  # set ticklines per tickvals array
            # 'tickvals': [0, 25, 75, 100],  # ticklines
            # 'tickvals': [25, 75],  # ticklines
            # 'type': 'log',
        },
        # panel mid
        "yaxis20": {
            "domain": [y20_btm, y20_top],
            "showticklabels": True,
            "showline": True,
            "title": mid_title,
            "titlefont": {"size": 12},
            "tickmode": "array",  # set ticklines per tickvals array
            # 'tickvals': [0, 25, 75, 100],  # ticklines
            # 'type': 'log',
        },
        # panel bottom
        "yaxis10": {
            "domain": [y10_btm, y10_top],
            "showticklabels": True,
            "showline": True,
            "title": btm_title,
            "titlefont": {"size": 12},
            # 'type': 'log',
            # 'tickmode': 'array',  # set ticklines per tickvals array
            # 'tickvals': [0, 25, 75, 100],  # ticklines
        },
    }

    # ++++++++++++ traces +++++++++++++
    data = []

    # spike line
    trace_spike_line = go.Scatter(
        # .astype(float) converts df.volume from integer to float for talib
        x=df.index,
        y=spike_line,
        yaxis="y80",
        line=dict(width=1),
        marker=dict(color=spike_line_color),
        hoverinfo="none",
        name="",
        showlegend=False,
    )
    data.append(trace_spike_line)

    # Candlestick Chart
    trace_price = go.Candlestick(
        x=df.index,
        # open=df.open,
        # high=df.high,
        # low=df.low,
        # close=df.close,
        open=df.Open,
        high=df.High,
        low=df.Low,        
        close=df.Close,    
        yaxis="y70",
        name=symbol,
        increasing=dict(line=dict(color=INCREASING_COLOR)),
        decreasing=dict(line=dict(color=DECREASING_COLOR)),
        showlegend=False,  # don't show legend across top of graph
        hoverinfo="all",  # display date, OHLC
    )
    data.append(trace_price)

    # Moving Averages
    trace_EMA_fast = go.Scatter(
        x=df.index,
        y=EMA_fast,
        yaxis="y70",
        name=EMA_fast_label,
        marker=dict(color=EMA_fast_color),
        hoverinfo="all",
        line=dict(width=1),
    )
    data.append(trace_EMA_fast)

    trace_EMA_slow = go.Scatter(
        x=df.index,
        y=EMA_slow,
        yaxis="y70",
        name=EMA_slow_label,
        marker=dict(color=EMA_slow_color),
        hoverinfo="all",
        line=dict(width=1),
    )
    data.append(trace_EMA_slow)

    # Bollinger Bands
    trace_BB_upper = go.Scatter(
        x=df.index,
        y=BB_upper,
        yaxis="y70",
        line=dict(width=1),
        marker=dict(color=BB_upper_color),
        hoverinfo="all",
        name=BBands_label,
        legendgroup="Bollinger Bands",
    )
    data.append(trace_BB_upper)

    trace_BB_avg = go.Scatter(
        x=df.index,
        y=BB_avg,
        yaxis="y70",
        line=dict(width=1),
        marker=dict(color=BB_avg_color),
        hoverinfo="all",
        name=BBands_label,
        showlegend=False,  # only show legend for upperband
        legendgroup="Bollinger Bands",
    )
    data.append(trace_BB_avg)

    trace_BB_lower = go.Scatter(
        x=df.index,
        y=BB_lower,
        yaxis="y70",
        line=dict(width=1),
        marker=dict(color=BB_lower_color),
        hoverinfo="all",
        name=BBands_label,
        showlegend=False,  # only show legend for upperband
        legendgroup="Bollinger Bands",
    )
    data.append(trace_BB_lower)

    # Volume Bars
    trace_vol = go.Bar(
        x=df.index,
        # y=df.volume,
        y=df.Volume,        
        marker=dict(color=colors_volume_bar),
        yaxis="y60",
        name="Volume",
        showlegend=False,
    )
    data.append(trace_vol)

    # panel traces
    drawdown_trace1 = go.Bar(
        x=df.index,
        y=DD_panel_trace1,
        yaxis="y50",
        # line=dict(width=1),
        marker=dict(color=DD_trace1_color),
        # hoverinfo='all',
        name=DD_label,
        showlegend=False,
    )
    data.append(drawdown_trace1)

    OBV_trace1 = go.Scatter(
        x=df.index,
        y=OBV_panel_trace1,
        yaxis="y40",
        line=dict(width=1),
        marker=dict(color=OBV_trace1_color),
        # hoverinfo='all',
        name=OBV_label,
        showlegend=False,
    )
    data.append(OBV_trace1)

    OBV_trace2 = go.Scatter(
        x=df.index,
        y=OBV_panel_trace2,
        yaxis="y40",
        # line=dict(width=1),
        line=dict(width=1.2),  # darker line for trace2
        marker=dict(color=OBV_trace2_color),
        # hoverinfo='all',
        name=OBV_EMA_label,
        showlegend=False,
    )
    data.append(OBV_trace2)

    top_trace1 = go.Bar(
        x=df.index,
        y=top_panel_trace1,
        yaxis="y30",
        # line=dict(width=1),
        marker=dict(color=colors_OBV_diff_bar),
        hoverinfo="all",
        name=top_panel_text1,
        showlegend=False,
    )
    data.append(top_trace1)

    mid_trace1 = go.Scatter(
        x=df.index,
        y=mid_panel_trace1,
        yaxis="y20",
        line=dict(width=1),
        marker=dict(color=mid_trace1_color),
        hoverinfo="all",
        name=mid_panel_text1,
        showlegend=False,
    )
    data.append(mid_trace1)

    mid_trace2 = go.Scatter(
        x=df.index,
        y=mid_panel_trace2,
        # yaxis='y20', line=dict(width=1),
        yaxis="y20",
        line=dict(width=1.2),  # darker line for trace2
        marker=dict(color=mid_trace2_color),
        hoverinfo="all",
        name=mid_panel_text2,
        showlegend=False,
    )
    data.append(mid_trace2)

    btm_trace1 = go.Scatter(
        x=df.index,
        y=btm_panel_trace1,
        yaxis="y10",
        line=dict(width=1),
        marker=dict(color=btm_trace1_color),
        hoverinfo="all",
        name=btm_panel_text1,
        showlegend=False,
    )
    data.append(btm_trace1)

    btm_trace2 = go.Scatter(
        x=df.index,
        y=btm_panel_trace2,
        # yaxis='y10', line=dict(width=1),
        yaxis="y10",
        line=dict(width=1.2),  # darker line for trace2
        marker=dict(color=btm_trace2_color),
        hoverinfo="all",
        name=btm_panel_text2,
        showlegend=False,
    )
    data.append(btm_trace2)

    if plot_chart:
        fig = go.Figure(data=data, layout=layout)
        plot(fig, filename=chart_path + chart_name)

    return cache


In [ ]:
for plt_sym in plt_syms:
# for plt_sym in plt_syms["VOO"]:  
  # yf_candlestick(plt_sym, df_adj_sunday[plt_sym].iloc[-days_plot:])
  yf_candlestick(plt_sym, df_adj[plt_sym].iloc[-days_plot:])  

In [3]:
def OBV_calc_orig(df, symbol, EMA_pd=50, tail_pd=5, norm_pd=30):
    """Calculate On-Balance-Volume(OBV), EMA of OBV,
       normalized EMA slope of the On-Balance-Volume (OBV).

    Args:
        df(dataframe): dataframe with date index
                       and columns open, high, low, close, volume
        symbol(str): symbol, e.g. 'QQQ'
        EMA_pd(int): exponential moving average period of OBV
        tail_pd(int): number of periods used to calculate avg_OBV_EMA_dif
        norm_pd(int): number of periods used to calculate avg_volume

    Return:
        OBV(numpy.ndarray): on-balance-volume
        OBV_EMA(numpy.ndarray): EMA of OBV with time-period=EMA_pd
        OBV_slope(float): avg_OBV_EMA_dif / avg_volume,
          (avg_OBV_EMA_dif is average difference of OBV_EMA the last tail_pd days, default is 5 days),
          (avg_volume is the average volume for last norm_pd days, default is 30 days)
    """
    # df.Close.values was df.close.values
    # df.Volume.values.astype(float) was df.volume.values.astype(float)
    # 2024-09-21 default EMA_pd was 10      

    import talib as ta
    import numpy as np


    close = df.Close.values  # convert dataframe to numpy array

    # .astype(float) converts df.volume from integer to float for talib
    # volume = df.volume.values.astype(float)
    volume = df.Volume.values.astype(float)

    OBV = ta.OBV(close, volume)

    OBV_EMA = ta.EMA(OBV, timeperiod=EMA_pd)  # calculate EMA of OBV

    # average OBV_EMA difference for the last tail_pd days, default is 5 days
    avg_OBV_EMA_dif = np.mean(np.diff(OBV_EMA[-(tail_pd+1):]))

    # average volume for last norm_pd days, default is 30 days
    avg_volume = np.mean(volume[-norm_pd:])

    # avg_OBV_EMA_dif divided by avg_volume
    OBV_slope = avg_OBV_EMA_dif / avg_volume


    # # print statements for debug
    # print('\n')
    # print('*'*20)
    # print('symbol: ', symbol)
    # print('avg_slope: ', avg_slope)
    # print('avg_volume: ', avg_volume)
    # print('OBV_slope: ', OBV_slope)
    # print('*'*20, '\n')

    return OBV, OBV_EMA, OBV_slope


In [ ]:
from langchain.text_splitter import SentenceTransformersTokenTextSplitter

character_split_texts = ["Dear shareholders, colleagues, customers, and partners"]

token_splitter = SentenceTransformersTokenTextSplitter(chunk_overlap=0, tokens_per_chunk=256)

token_split_texts = []
for text in character_split_texts:
    token_split_texts += token_splitter.split_text(text)

print(f"len(token_split_texts): {len(token_split_texts)}")
print(f"len(token_split_texts[0]): {len(token_split_texts[0])}")

In [4]:
from langchain.text_splitter import SentenceTransformersTokenTextSplitter

In [ ]:
_texts = "Dear shareholders, colleagues, customers, and partners:  We are living through a period of historic economic, societal, and geopolitical change. The world in 2022 looks nothing like the world in 2019. As I write this, inflation is at a 40 -year high, supply chains are stretched, and the war in Ukraine is ongoing. At the same time, we are entering a technological era with the potential to power awesome advancements across every sector of our economy and society. As the world's largest software company, this places us at a historic intersection of opportunity and responsibility to the world around us. Our mission to empower every person and every organization on the planet to achieve more has never been more urgent or more necessary. For all the uncertainty in the world, one thing is clear: People and organizations in every industry are increasingly looking to digital technology to overcome today's challenges and emerge stronger. And no"
print(f"len(_texts): {len(_texts)}")

In [21]:
character_split_texts = ["Dear shareholders, colleagues, customers, and partners:  We are living through a period of historic economic, societal, and geopolitical change. The world in 2022 looks nothing like the world in 2019. As I write this, inflation is at a 40 -year high, supply chains are stretched, and the war in Ukraine is ongoing. At the same time, we are entering a technological era with the potential to power awesome advancements across every sector of our economy and society. As the world's largest software company, this places us at a historic intersection of opportunity and responsibility to the world around us. Our mission to empower every person and every organization on the planet to achieve more has never been more urgent or more necessary. For all the uncertainty in the world, one thing is clear: People and organizations in every industry are increasingly looking to digital technology to overcome today's challenges and emerge stronger. And no"]
# character_split_texts = ["Dear shareholders, colleagues, customers, and partners"]

In [22]:
# SentenceTransformers context windows is 256 max 
token_splitter = SentenceTransformersTokenTextSplitter(chunk_overlap=0, tokens_per_chunk=256)

token_split_texts = []
for text in character_split_texts:
    token_split_texts += token_splitter.split_text(text)

In [ ]:
len(token_split_texts[0])